Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Multi Class Perceptron

In [5]:
class MultiClassPerceptron:
    """
    Implementation of a multi-class perceptron as a weak learner.
    Supports weighted samples for integration with ensemble methods like Adaboost.
    """

    def __init__(self, alpha=0.1, n_iterations=100):
        """
        Initialize the perceptron with learning rate and number of iterations.

        Args:
            alpha (float): Learning rate for weight updates.
            n_iterations (int): Maximum number of iterations for training.
        """
        self.alpha = alpha
        self.n_iterations = n_iterations
        self.weights = {}
        self.biases = {}

    def train(self, X, y, sample_weights=None, weak_threshold=0.55):
        """
        Train the perceptron for multi-class classification.

        Args:
            X (np.ndarray): Training data, shape (n_samples, n_features).
            y (np.ndarray): Training labels, shape (n_samples,).
            sample_weights (np.ndarray, optional): Weights for training samples.
            weak_threshold (float): Minimum accuracy to stop training early.

        Returns:
            self: Trained MultiClassPerceptron instance.
        """
        self.classes = np.unique(y)
        n_samples, n_features = X.shape

        # Initialize sample weights if not provided
        if sample_weights is None:
            sample_weights = np.ones(n_samples) / n_samples

        for cls in self.classes:
            # Initialize weights and biases for the current class
            self.weights[cls] = np.zeros(n_features)
            self.biases[cls] = 0

            # Create binary labels for the current class (1 for cls, -1 for others)
            y_binary = np.where(y == cls, 1, -1)

            # Training loop
            for _ in range(self.n_iterations):
                correct_predictions = 0
                for idx in range(n_samples):
                    # Predict using the current weights and bias
                    y_pred = np.sign(np.dot(self.weights[cls], X[idx]) + self.biases[cls])
                    if y_pred == y_binary[idx]:
                        correct_predictions += 1
                    else:
                        # Update weights and bias if prediction is incorrect
                        update = self.alpha * y_binary[idx] * sample_weights[idx]
                        self.weights[cls] += update * X[idx]
                        self.biases[cls] += update

                # Stop training early if weak threshold is met
                accuracy = correct_predictions / n_samples
                if accuracy > weak_threshold:
                    break

        return self

    def predict(self, X):
        """
        Predict class labels for input data.

        Args:
            X (np.ndarray): Input data, shape (n_samples, n_features).

        Returns:
            np.ndarray: Predicted class labels, shape (n_samples,).
        """
        scores = {cls: np.dot(X, self.weights[cls]) + self.biases[cls] for cls in self.classes}
        return np.array([self.classes[np.argmax([scores[cls][i] for cls in self.classes])] for i in range(X.shape[0])])

    def score(self, X, y):
        """
        Compute the accuracy of the perceptron.

        Args:
            X (np.ndarray): Input data.
            y (np.ndarray): True labels.

        Returns:
            float: Accuracy score.
        """
        return np.mean(self.predict(X) == y)

Multi-class Adaboost Algorithm :

In [7]:
class MultiClassAdaboost:
    """
    Implementation of the Multi-Class Adaboost algorithm.
    """

    def __init__(self, weak_learner=MultiClassPerceptron, n_estimators=50):
        """
        Initialize the Adaboost ensemble with weak learners.

        Args:
            weak_learner (class): Weak learner class (default: MultiClassPerceptron).
            n_estimators (int): Number of weak learners to train.
        """
        self.n_estimators = n_estimators
        self.weak_learner = weak_learner
        self.weak_learners = []
        self.alphas = []
        self.classes = None

    def train(self, X, y):
        """
        Train the Adaboost ensemble.

        Args:
            X (np.ndarray): Training data, shape (n_samples, n_features).
            y (np.ndarray): Training labels, shape (n_samples,).

        Returns:
            self: Trained MultiClassAdaboost instance.
        """
        self.classes = np.unique(y)
        n_samples, n_features = X.shape

        # Initialize sample weights
        sample_weights = np.ones(n_samples) / n_samples

        for t in range(self.n_estimators):
            # Train a weak learner
            weak_learner = self.weak_learner().train(X, y, sample_weights)
            self.weak_learners.append(weak_learner)

            # Get predictions and calculate weighted error
            weak_predictions = weak_learner.predict(X)
            incorrect = (weak_predictions != y)
            weighted_error = np.sum(sample_weights * incorrect)

            # Stop if the weak learner is no better than random guessing
            if weighted_error >= 0.5:
                break

            # Calculate alpha (importance of the weak learner)
            alpha = 0.5 * np.log((1 - weighted_error) / (weighted_error + 1e-10))
            self.alphas.append(alpha)

            # Update sample weights
            sample_weights *= np.exp(alpha * incorrect)
            sample_weights /= np.sum(sample_weights)

        return self

    def predict(self, X):
        """
        Predict class labels using the ensemble of weak learners.

        Args:
            X (np.ndarray): Input data, shape (n_samples, n_features).

        Returns:
            np.ndarray: Predicted class labels, shape (n_samples,).
        """
        votes = np.zeros((X.shape[0], len(self.classes)))

        for learner, alpha in zip(self.weak_learners, self.alphas):
            predictions = learner.predict(X)
            for i, pred in enumerate(predictions):
                class_idx = np.where(self.classes == pred)[0][0]
                votes[i, class_idx] += alpha

        return self.classes[np.argmax(votes, axis=1)]

    def score(self, X, y):
        """
        Compute the accuracy of the Adaboost model.

        Args:
            X (np.ndarray): Input data.
            y (np.ndarray): True labels.

        Returns:
            float: Accuracy score.
        """
        return np.mean(self.predict(X) == y)


Main script for data loading, preprocessing, training, and evaluation

In [8]:
# Load the dataset
digits = load_digits()
X, y = digits.data, digits.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Multi-Class Adaboost
adaboost = MultiClassAdaboost(n_estimators=50)
adaboost.train(X_train_scaled, y_train)

# Evaluate the model
y_pred = adaboost.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9278

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        36
           1       0.90      0.75      0.82        36
           2       0.94      0.97      0.96        35
           3       0.97      0.92      0.94        37
           4       0.97      0.97      0.97        36
           5       0.97      0.95      0.96        37
           6       1.00      0.92      0.96        36
           7       0.97      1.00      0.99        36
           8       0.81      0.86      0.83        35
           9       0.79      0.94      0.86        36

    accuracy                           0.93       360
   macro avg       0.93      0.93      0.93       360
weighted avg       0.93      0.93      0.93       360


Confusion Matrix:
[[36  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  1  1  0  0  0  3  4]
 [ 0  1 34  0  0  0  0  0  0  0]
 [ 0  0  2 34  0  0  0  0  0  1]
 [ 0  0  0  0 35  0  0  0  0  1]
 [ 0  0  0  0  0